# Atividade 01

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os monitores
- Divirta-se :)

### Baixando dados de ativos

1. Baixe os dados de três ativos da sua escolha do Yahoo Finance, correspondente a um período de pelo menos 2 anos até a data mais recente.

2. Salve esses dados para que você possa utilizá-los na próxima tarefa.

3.  Observando a volatilidade dentro do dia

4. Calcule a amplitude de variação do valor do ativo dentro do dia. Essa amplitude você pode calcular com um comando simples de adicionar uma coluna no Pandas: (valor máximo - valor mínimo).

5. Monte um gráfico de linha mostrando a variação dessa amplitude ao longo do período coletado, por ativo.

6. Veja se consegue identificar que ativo apresentou maior volatilidade.

In [46]:
#Importando bibliotecas
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
import warnings
from prettytable import PrettyTable

In [54]:
def plot_stock_amplitude(tickers, start_date, end_date, figsize=(800, 600)):
    # Ignorar FutureWarnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    # Armazenar todos os dados das ações em um único DataFrame
    all_data = pd.DataFrame()
    traces, trendlines, min_max_points = [], [], []

    # Criar uma tabela para armazenar os resultados
    result_table = PrettyTable()
    result_table.field_names = ["Ação", "Amplitude Mínima", "Data (Mínima)", "Amplitude Máxima", "Data (Máxima)"]

    # Loop sobre todas as ações
    for stock in tickers:
        # Baixar dados da ação
        data = yf.download(stock, start=start_date, end=end_date)
        data.reset_index(inplace=True)
        
        # Calcular amplitude diária
        data['Amplitude'] = data['High'] - data['Low']
        
        # Encontrar a data e o valor mínimos da amplitude
        min_date = data.loc[data['Amplitude'].idxmin(), 'Date']
        min_value = round(data['Amplitude'].min(), 2)
        
        # Encontrar a data e o valor máximos da amplitude
        max_date = data.loc[data['Amplitude'].idxmax(), 'Date']
        max_value = round(data['Amplitude'].max(), 2)
        
        # Adicionar os resultados à tabela
        result_table.add_row([stock, min_value, min_date, max_value, max_date])

        # Criar traces para plotagem
        trace = go.Scatter(x=data['Date'], y=data['Amplitude'], mode='lines+markers', name=stock)
        trendline = go.Scatter(x=data['Date'], y=data['Amplitude'].rolling(window=30).mean(), mode='lines', name=f'Trend - {stock}')
        min_point = go.Scatter(x=[min_date], y=[min_value], mode='markers', marker=dict(color='red'), name=f'Min - {stock}')
        max_point = go.Scatter(x=[max_date], y=[max_value], mode='markers', marker=dict(color='green'), name=f'Max - {stock}')
        
        # Adicionar traces às listas correspondentes
        traces.append(trace)
        trendlines.append(trendline)
        min_max_points.extend([min_point, max_point])
        
        # Adicionar dados da ação ao DataFrame geral
        all_data = pd.concat([all_data, data.set_index('Date')], axis=1)

    # Imprimir a tabela formatada
    print(result_table)

    # Salvar dados das ações em um arquivo CSV
    all_data.to_csv('ativos.csv')
    print("Dados das ações baixados e salvos com sucesso!")

    # Definir layout do gráfico
    layout = go.Layout(
        title=dict(text='Variação da Amplitude do Preço das Ações Dentro de um Dia', font=dict(color='#FFFFFF', size=20)),
        xaxis=dict(title='Data', tickangle=45, tickfont=dict(color='#FFFFFF', size=14), titlefont=dict(color='#FFFFFF', size=16)),
        yaxis=dict(title='Variação da Amplitude', tickfont=dict(color='#FFFFFF', size=14), titlefont=dict(color='#FFFFFF', size=16)),
        showlegend=True,
        plot_bgcolor='rgb(0, 0, 0)',
        paper_bgcolor='rgb(0, 0, 0)',
        legend=dict(font=dict(color='#FFFFFF')),
        width=figsize[0],
        height=figsize[1],
        xaxis_showgrid=False,
        yaxis_showgrid=False
    )

    # Criar figura do gráfico
    fig = go.Figure(data=traces + trendlines + min_max_points, layout=layout)
    fig.show()

# Ações e período para análise
tickers = ['NVDA', 'MSFT', 'AAPL']
start_date = '2022-01-01'
end_date = '2024-02-09'
figsize = (1000, 500)

# Plotar amplitude das ações
plot_stock_amplitude(tickers, start_date, end_date, figsize)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

+------+------------------+---------------------+------------------+---------------------+
| Ação | Amplitude Mínima |    Data (Mínima)    | Amplitude Máxima |    Data (Máxima)    |
+------+------------------+---------------------+------------------+---------------------+
| NVDA |       3.15       | 2022-11-25 00:00:00 |      34.89       | 2023-08-14 00:00:00 |
| MSFT |       1.97       | 2022-11-25 00:00:00 |      24.61       | 2023-07-18 00:00:00 |
| AAPL |       1.06       | 2023-12-19 00:00:00 |      10.85       | 2022-02-24 00:00:00 |
+------+------------------+---------------------+------------------+---------------------+
Dados das ações baixados e salvos com sucesso!


- Sim, NVDA apresentou a maior volatilidade no período.

# Atividade 02

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os monitores
- Divirta-se :)

#### 1. Carregue os dados da tarefa anterior

In [48]:
# Carregar dados das ações
all_data = pd.read_csv('ativos.csv', index_col='Date', parse_dates=True)

#### 2. Média móvel

1. Faça uma média móvel  em 90 dias dos valores de fechamento ajustados dos ativos que você escolheu
2. Utilizando a fução ```rolling()```, calcule o desvio pardão desses ativos ao longo do tempo, utilizando a mesma janela móvel do item acima
3. Monte um gráfico de linhas para as médias móveis dos ativos da sua base
4. Monte um gráfico de linhas para os desvios padrão em janela móvel
5. Compare os dois gráficos que você fez com o da tarefa anterior. Veja se eles apontam conclusões semelhantes quanto aos períodos de maior volatilidade (variação) dos ativos.

In [49]:
# Calcular média móvel de 90 dias dos valores de fechamento ajustados das ações
moving_average = all_data['Adj Close'].rolling(window=90).mean()

# Criar tabela para as médias móveis
table = PrettyTable()
table.field_names = ["Data", "Média Móvel"]
for i in range(5):
    table.add_row([moving_average.index[i], moving_average.iloc[i]])

# Imprimir a tabela
print("Médias móveis:")
print(table)

# Arredondar o valor da média móvel no dia 90
rounded_value = round(moving_average.iloc[89], 2)

# Criar e imprimir tabela para o valor da média móvel no dia 90
print(PrettyTable([["Dia", "Valor da Média Móvel"], ["Dia 90", rounded_value]]))

Médias móveis:
+---------------------+-------------+
|         Data        | Média Móvel |
+---------------------+-------------+
| 2022-01-03 00:00:00 |     nan     |
| 2022-01-04 00:00:00 |     nan     |
| 2022-01-05 00:00:00 |     nan     |
| 2022-01-06 00:00:00 |     nan     |
| 2022-01-07 00:00:00 |     nan     |
+---------------------+-------------+
+---------------------------------+--------------------+
| ['Dia', 'Valor da Média Móvel'] | ['Dia 90', 237.73] |
+---------------------------------+--------------------+
+---------------------------------+--------------------+


In [50]:
# Calcular desvio padrão em janela móvel (90 dias) dos valores de fechamento ajustados das ações
std_deviation = all_data['Adj Close'].rolling(window=90).std()

# Criar tabela para os desvios padrão
table = PrettyTable()
table.field_names = ["Data", "Desvio Padrão"]
for i in range(5):
    table.add_row([std_deviation.index[i], std_deviation.iloc[i]])

# Imprimir a tabela
print("Desvios padrão:")
print(table)

# Arredondar o valor do desvio padrão no dia 90
rounded_value = round(std_deviation.iloc[89], 2)

# Criar e imprimir tabela para o valor do desvio padrão no dia 90
print(PrettyTable([["Dia", "Valor do Desvio Padrão"], ["Dia 90", rounded_value]]))

Desvios padrão:
+---------------------+---------------+
|         Data        | Desvio Padrão |
+---------------------+---------------+
| 2022-01-03 00:00:00 |      nan      |
| 2022-01-04 00:00:00 |      nan      |
| 2022-01-05 00:00:00 |      nan      |
| 2022-01-06 00:00:00 |      nan      |
| 2022-01-07 00:00:00 |      nan      |
+---------------------+---------------+
+-----------------------------------+-------------------+
| ['Dia', 'Valor do Desvio Padrão'] | ['Dia 90', 30.37] |
+-----------------------------------+-------------------+
+-----------------------------------+-------------------+


In [51]:
# Criar e exibir gráfico de linha para médias móveis das ações
fig_moving_average = go.Figure(go.Scatter(x=moving_average.index,
                                          y=moving_average,
                                          mode='lines',
                                          name='Média Móvel'))
fig_moving_average.update_layout(title='Média Móvel de 90 Dias das Ações',
                                 xaxis_title='Data',
                                 yaxis_title='Média Móvel')
fig_moving_average.show()

In [52]:
# Criar e exibir gráfico de linha para desvios padrão na janela móvel
fig_std_deviation = go.Figure(go.Scatter(x=std_deviation.index,
                                         y=std_deviation,
                                         mode='lines',
                                         name='Desvio Padrão'))
fig_std_deviation.update_layout(title='Desvio Padrão na Janela Móvel (90 dias) das Ações',
                                xaxis_title='Data',
                                yaxis_title='Desvio Padrão')
fig_std_deviation.show()

- A volatilidade das ações variou ao longo do tempo, sendo maior para a NVDA. De fato, o movimento da NVDA foi tão destacado que influenciou no fechamento dos três ativos, definindo as tendências, uma vez que estes não apresentaram movimentos parecidos aos da fabricante de GPUs. Talvez seja uma boa comprar NVDA.

#### 3. Gráfico de *candlestick*
1. Selecione o período dos últimos 60 dias (corridos) para um dos ativos da sua base
2. Monte um gráfico de *candlestick* para esses dados.

In [53]:
# Definir o ticker NVDA e o período de interesse
ticker_nvda = 'NVDA'
start_date = '2022-01-01'
end_date = '2024-02-09'

# Baixar dados para o ticker NVDA do Yahoo Finance e selecionar o período dos últimos 60 dias
nvda_last_60_days = yf.download(ticker_nvda, start=start_date, end=end_date).tail(60)

# Criar e exibir gráfico de candlestick
fig = go.Figure(data=[go.Candlestick(x=nvda_last_60_days.index,
                                     open=nvda_last_60_days['Open'],
                                     high=nvda_last_60_days['High'],
                                     low=nvda_last_60_days['Low'],
                                     close=nvda_last_60_days['Close'])])
fig.update_layout(title='Gráfico de Candlestick para NVDA (Últimos 60 dias)',
                  xaxis_title='Data',
                  yaxis_title='Preço',
                  xaxis_rangeslider_visible=False)
fig.show()

[*********************100%%**********************]  1 of 1 completed
